In [1]:
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
import numpy as np

In [3]:
data_path = Path("../data/raw")

In [26]:
train = pd.read_csv(data_path/"train.csv", sep=";")
test = pd.read_csv(data_path/"test.csv", sep=";")
edu = pd.read_csv(data_path/"education.csv", sep=";")
empl = pd.read_csv(data_path/"employements.csv", sep=";")
world = pd.read_csv(data_path/"worldskills.csv", sep=";")
edu_mult = pd.read_csv(data_path/"education_mult.csv", sep=";")
empl_mult = pd.read_csv(data_path/"employements_mult.csv", sep=";")
world_mult = pd.read_csv(data_path/"worldskills_mult.csv", sep=";")

/home/alexander/anaconda3/envs/pand/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## train date features

In [12]:
train["creation_date"] = pd.to_datetime(train["creation_date"])

In [16]:
train["creation_date"].dt.month

0          5.0
1         10.0
2          4.0
3          9.0
4          4.0
          ... 
306265     5.0
306266     4.0
306267    11.0
306268     6.0
306269     4.0
Name: creation_date, Length: 306270, dtype: float64

In [17]:
new_time_features = pd.DataFrame()

In [ ]:
for col in date_cols:
    new_time_features[f"col_day"] = train[col].dt.day
    new_time_features[f"col_month"] = train[col].dt.month
    new_time_features[f"col_year"] = train[col].dt.year

In [25]:
((pd.to_datetime(train["creation_date"]) - pd.to_datetime(train["modification_date"])).dt.days).abs()

0           1.0
1           6.0
2         285.0
3           1.0
4         142.0
          ...  
306265      3.0
306266      3.0
306267    906.0
306268      1.0
306269      9.0
Length: 306270, dtype: float64

In [35]:
def date_preprocess(train):
    train = train.copy()
    date_cols = ["creation_date", "modification_date", "publish_date"]
    for col in date_cols:
        train[col] = pd.to_datetime(train[col], format="%Y-%m-%d")
    new_time_features = pd.DataFrame()
    for col in date_cols:
        new_time_features[f"{col}_day"] = train[col].dt.day
        new_time_features[f"{col}_month"] = train[col].dt.month
        new_time_features[f"{col}_year"] = train[col].dt.year
        
    for i in range(len(date_cols)):
        for j in range(i, len(date_cols)):
            col1 = date_cols[i]
            col2 = date_cols[j]
            new_time_features[f"{col1}_minus_{col2}"] = (train[col1] - train[col2]).dt.days.abs()
    return new_time_features

In [36]:
res = date_preprocess(train)

In [37]:
res.head()

,creation_date_day,creation_date_month,creation_date_year,modification_date_day,modification_date_month,modification_date_year,publish_date_day,publish_date_month,publish_date_year,creation_date_minus_creation_date,creation_date_minus_modification_date,creation_date_minus_publish_date,modification_date_minus_modification_date,modification_date_minus_publish_date,publish_date_minus_publish_date
0,7.0,5.0,2020.0,8.0,5.0,2020.0,7,5,2020,0.0,1.0,0.0,0.0,1.0,0
1,21.0,10.0,2020.0,27.0,10.0,2020.0,27,10,2020,0.0,6.0,6.0,0.0,0.0,0
2,25.0,4.0,2019.0,4.0,2.0,2020.0,4,2,2020,0.0,285.0,285.0,0.0,0.0,0
3,19.0,9.0,2020.0,20.0,9.0,2020.0,19,9,2020,0.0,1.0,0.0,0.0,1.0,0
4,20.0,4.0,2020.0,9.0,9.0,2020.0,21,4,2020,0.0,142.0,1.0,0.0,141.0,0


In [38]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306270 entries, 0 to 306269
Data columns (total 15 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   creation_date_day                          306263 non-null  float64
 1   creation_date_month                        306263 non-null  float64
 2   creation_date_year                         306263 non-null  float64
 3   modification_date_day                      306263 non-null  float64
 4   modification_date_month                    306263 non-null  float64
 5   modification_date_year                     306263 non-null  float64
 6   publish_date_day                           306270 non-null  int64  
 7   publish_date_month                         306270 non-null  int64  
 8   publish_date_year                          306270 non-null  int64  
 9   creation_date_minus_creation_date          306263 non-null  float64
 10  creation

In [44]:
train[date_cols[2]].isna().sum()

0

## EDA

### train

In [6]:
train_cat = [
    "region",
    "industry",
    "education_type",
    "drive_licences", # 
    "citizenship",
    "schedule",
    "employement_type",
    "gender"
]

train_binary = [
    "relocation_ready",
    "travel_ready",
    "retraining_ready",
    "is_worldskills_participant",
    "has_qualifications"
]

train_text = [
    "position",
]

train_float = [
    "age",
    "experience",
    "salary_desired", #
    "completeness_rate"
]

train_idk = [
    "locality",
    "locality_name"
]

train_dates = [
    "creation_date",
    "modification_date",
    "publish_date"
]

In [10]:
(set(train_binary) | set(train_cat) | set(train_text) | set(train_float) | set(train_idk) | set(train_dates) ).symmetric_difference(set(train.columns))

{'id', 'salary'}

In [11]:
for col in train.columns:
    print(col, train[col].nunique())

id 306270
position 77762
region 86
industry 33
locality 12365
locality_name 8684
education_type 4
drive_licences 29
citizenship 12
schedule 63
employement_type 6
age 79
gender 2
experience 73
salary_desired 2046
relocation_ready 2
travel_ready 2
retraining_ready 2
is_worldskills_participant 2
has_qualifications 1
completeness_rate 57
creation_date 1504
modification_date 579
publish_date 1893
salary 19203


In [12]:
train.head()

,id,position,region,industry,locality,locality_name,education_type,drive_licences,citizenship,schedule,employement_type,age,gender,experience,salary_desired,relocation_ready,travel_ready,retraining_ready,is_worldskills_participant,has_qualifications,completeness_rate,creation_date,modification_date,publish_date,salary
0,0,Специалист,Тульская область,"Государственная служба, некоммерческие организ...",7100000100000,Тула,Незаконченное высшее,[B],Российская Федерация,Полный рабочий день,Полная занятость,21.0,Мужской,3,35000,False,False,True,NaN,NaN,64.0,2020-05-07,2020-05-08,2020-05-07,37500
1,1,Лаборант,Алтайский край,"Государственная служба, некоммерческие организ...",2200000100000,Барнаул,Высшее,[B],Российская Федерация,Полный рабочий день,Полная занятость,26.0,Женский,4,15000,False,True,True,NaN,NaN,88.0,2020-10-21,2020-10-27,2020-10-27,14000
2,3,"Специалист, администратор, бухгалтер, экономист",Ульяновская область,"Административная работа, секретариат, АХО",7300000200000,Димитровград,Высшее,[B],Российская Федерация,Полный рабочий день,Полная занятость,36.0,Женский,12,20000,False,NaN,True,NaN,NaN,78.0,2019-04-25,2020-02-04,2020-02-04,24600
3,4,Специалист,Свердловская область,"Государственная служба, некоммерческие организ...",6600002300000,Нижний Тагил,Высшее,[B],Российская Федерация,Полный рабочий день,Полная занятость,34.0,Женский,9,30000,False,False,False,NaN,NaN,91.0,2020-09-19,2020-09-20,2020-09-19,16075
4,5,Инструктор по плаванию,Иркутская область,"Здравоохранение, спорт, красота, социальное об...",3800000400000,Ангарск,NaN,NaN,Российская Федерация,Полный рабочий день,Полная занятость,25.0,Мужской,1,40000,False,NaN,NaN,NaN,NaN,58.0,2020-04-20,2020-09-09,2020-04-21,22080


In [11]:
for col in train.columns:
    print(col, len(train[col]) - len(train[col].dropna()))

id 0
position 0
region 12
industry 0
locality 0
locality_name 15492
education_type 40032
drive_licences 184345
citizenship 0
schedule 0
employement_type 0
age 7811
gender 7773
experience 0
salary_desired 0
relocation_ready 72703
travel_ready 69216
retraining_ready 57126
is_worldskills_participant 306076
has_qualifications 303736
completeness_rate 0
creation_date 7
modification_date 7
publish_date 0
salary 0


In [12]:
len(train)

306270

### world

In [14]:
print(len(world))
world.head()

1322


,id,status,code,is_international,int_name,ru_name
0,166,Данные подтверждены,R4,False,Preschool Education,Дошкольное воспитание
1,425,Данные в процессе проверки,17,True,Web Design and Development,Веб-дизайн и разработка
2,498,Данные в процессе проверки,33,True,Automobile Technology,Ремонт и обслуживание легковых автомобилей
3,596,Данные в процессе проверки,R45,False,NaN,Специалист по организационному и документацион...
4,1256,Данные в процессе проверки,NaN,NaN,NaN,NaN


In [21]:
for col in world.columns:
    print(col, world[col].nunique())

id 1322
status 2
code 199
is_international 2
int_name 132
ru_name 199


In [20]:
for col in world.columns:
    print(col, len(world[col]) - len(world[col].dropna()))

id 0
status 0
code 335
is_international 314
int_name 427
ru_name 314


In [16]:
len(world)

1322

### edu

In [22]:
edu.head()

,id,graduation_year,institution,description
0,0,2022.0,"""Тульский государственный университет""",NaN
1,1,2016.0,"""Алтайский государственный аграрный университет""",NaN
2,2,2014.0,"""Алтайский государственный технический универс...",NaN
3,3,2003.0,ГОУ СПО ДИМИТРОВГРАДСКИЙ ТЕХНИЧЕСКИЙ КОЛЛЕДЖ,NaN
4,4,2005.0,Нижнетагильский торгово-экономический техникум,NaN


In [25]:
len(edu)

375364

In [24]:
for col in edu.columns:
    print(col, edu[col].nunique())

id 375364
graduation_year 90
institution 151226
description 34


In [23]:
for col in edu.columns:
    print(col, len(edu[col]) - len(edu[col].dropna()))

id 0
graduation_year 1
institution 362
description 375299


### edu_mult

In [27]:
edu_mult.head()

,id,graduation_year,institution,description
0,0.0,2022.0,"""Тульский государственный университет""",NaN
1,1.0,2016.0,"""Алтайский государственный аграрный университет""",NaN
2,10.0,2018.0,Байкальский Государственный Университет г. Ирк...,NaN
3,10.0,2014.0,якутский колледж инновационных технологий,NaN
4,100.0,2009.0,"""Московский гуманитарный университет""",NaN


In [28]:
len(edu_mult)

453210

In [29]:
for col in edu.columns:
    print(col, edu[col].nunique())

id 375364
graduation_year 90
institution 151226
description 34


In [30]:
for col in edu.columns:
    print(col, len(edu[col]) - len(edu[col].dropna()))

id 0
graduation_year 1
institution 362
description 375299
